In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.preprocessing import LabelEncoder
import sklearn
from tqdm.notebook import tqdm

y_label = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
X_train = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
X_test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
sumple_submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

In [ ]:
train = pd.merge(X_train,y_label,on="sig_id")
X_train = X_train.drop("sig_id",axis=1)
y_label = y_label.drop("sig_id",axis=1)

In [ ]:
cp_time = {24: 1,48: 2, 72:3}
cp_dose = {'D1': 1,'D2': 2}
cp_type = {'trt_cp':1, 'ctl_vehicle':0 }


X_train['cp_time'] = X_train['cp_time'].map(cp_time)
X_train['cp_dose'] = X_train['cp_dose'].map(cp_dose)
X_train['cp_type'] = X_train['cp_type'].map(cp_type)

X_test['cp_time'] = X_test['cp_time'].map(cp_time)
X_test['cp_dose'] = X_test['cp_dose'].map(cp_dose)
X_test['cp_type'] = X_test['cp_type'].map(cp_type)

# **The below code transformes the mutilabel problem into multiclass**

In [ ]:
col = np.array(y_label.columns)
X = np.empty(shape=X_train.shape)
Y = []
temp = y_label.values
for i,v in tqdm(enumerate(X_train.values)):
    index = [k for k, i in enumerate(temp[i]) if i == 1] #checks if label contains 1
    X[i] = v
    if index:
        Y.append(":".join(col[index]))
    else:
        Y.append("no")
print(len(set(Y)))
X[:3,:5],Y[0]

Pay attention on the last column which is the multiclass label

In [ ]:
target = dict(zip(list(set(Y)),range(len(set(Y)))))
rev = dict(zip(range(len(set(Y))),list(set(Y))))
len(target)

In [ ]:
def get_target(value):
    if ":" in value:
        out = np.zeros(col.shape)
        for v in value.split(":"):
            out[np.where(col==v)[0]] = 1.0
        return out
    elif value == "no":
        return np.zeros(col.shape)
    else:
        out = np.zeros(col.shape)
        out[np.where(col==value)[0]]=1.0
        return out

In [ ]:
y = pd.Series(Y).replace(target)
X.shape,y.shape

In [ ]:
del y_label,X_train,X_test,train
import gc
gc.collect()

In [ ]:
# BaggingClassifier(base_estimator=SVC(),n_estimators=10, random_state=0)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import tensorflow as tf
kf = StratifiedKFold(n_splits=50)
log_loss = []
for train_index, test_index in kf.split(X,y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = SVC()
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    y_p = np.empty((y_test.shape[0],206))
    y_t = np.empty((y_test.shape[0],206))
    y_test = y_test.reset_index(drop=True)
    print(accuracy_score(y_test,pred))
    #print(log_loss(y_test,pred))
    for i,v in y_test.iteritems():
        y_t[i] = get_target(rev[v])
        y_p[i] = get_target(rev[pred[i]])
    loss = tf.keras.losses.binary_crossentropy(y_t, y_p).numpy().mean()
    print(loss)
    log_loss.append(loss)
np.mean(log_loss)

In [ ]:
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,ExtraTreesClassifier,GradientBoostingClassifier,StackingClassifier,BaggingClassifier
# estimators = [('rf', RandomForestClassifier(n_estimators=10, random_state=42)),("svc",SVC())]
# clf = StackingClassifier(estimators=estimators, final_estimator=RandomForestClassifier())

In [ ]:
# clf.fit(X,y)   

In [ ]:
import pickle
filename = 'finalized_model.pkl'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
#model.predit(X)

In [ ]:
#pred.to_csv('submission.csv', index = False)